# Perform Necessary Imports

In [1]:
import pandas as pd
import xgboost
import re
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
from keras.preprocessing import text, sequence
import numpy as np


In [2]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.base import TransformerMixin
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [3]:
class MeanEmbeddingTransformer(TransformerMixin):
    
    def __init__(self):
        self._vocab, self._E = self._load_words()
        
    
    def _load_words(self):
        E = {}
        vocab = []

        with open('glove.6B.50d.txt', 'r', encoding="utf8") as file:
            for i, line in enumerate(file):
                l = line.split(' ')
                if l[0].isalpha():
                    v = [float(i) for i in l[1:]]
                    E[l[0]] = np.array(v)
                    vocab.append(l[0])
        return np.array(vocab), E            

    
    def _get_word(self, v):
        for i, emb in enumerate(self._E):
            if np.array_equal(emb, v):
                return self._vocab[i]
        return None
    
    def _doc_mean(self, doc):
        return np.mean(np.array([self._E[w.lower().strip()] for w in doc if w.lower().strip() in self._E]), axis=0)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([self._doc_mean(doc) for doc in X])
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

# Read filtered Dataset

In [4]:
totalData = pd.read_csv('../datasets/nela10.csv')

# How it looks like?


In [5]:
totalData

,id,date,source,title,content,author,url,published,published_utc,collection_utc,Reliability
0,abcnews--2019-01-30--Who is Stacey Abrams the ...,2019-01-30,abcnews,"Who is Stacey Abrams, the Democrat who will re...","Last November, Stacey Abrams, a Democrat who r...",Cheyenne Haslett,https://abcnews.go.com/Politics/stacey-abrams-...,2019-01-30 19:10:47+00:00,1548893447,1567550233,0
1,abcnews--2019-01-30--Ban government shutdowns ...,2019-01-30,abcnews,Ban government shutdowns? Some Republicans and...,As lawmakers scramble to negotiate a border se...,"Benjamin Siegel,\nTrish Turner\n",https://abcnews.go.com/Politics/ban-government...,2019-01-30 17:29:04+00:00,1548887344,1567550233,0
2,abcnews--2019-01-30--Democrats renew push for ...,2019-01-30,abcnews,Democrats renew push for equal pay for equal work,Ten years after President Barack Obama signed ...,John Parkinson,https://abcnews.go.com/Politics/dems-renew-pus...,2019-01-30 21:35:45+00:00,1548902145,1567550233,0
3,abcnews--2019-01-30--Trump attacks intel chief...,2019-01-30,abcnews,Trump attacks intel chiefs after they contradi...,"President Donald Trump on Wednesday declared ""...",Meridith Mcgraw,https://abcnews.go.com/Politics/trump-attacks-...,2019-01-30 17:03:40+00:00,1548885820,1567550233,0
4,abcnews--2019-01-30--Border security battle be...,2019-01-30,abcnews,Border security battle begins as Congress grap...,Republicans and Democrats on the exclusive com...,"John Parkinson,\nBenjamin Siegel\n",https://abcnews.go.com/Politics/border-securit...,2019-01-30 15:09:47+00:00,1548878987,1567550233,0
...,...,...,...,...,...,...,...,...,...,...,...
57152,zerohedge--2019-11-01--Hong Kong Officer Faces...,2019-11-01,zerohedge,Hong Kong Officer Faces Death Threats After Fi...,It's unclear exactly when it happened (the BBG...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 22:45:00 +0000",1572662700,1572648625,2
57153,"zerohedge--2019-11-01--Enough ""Quid Pro Quo"" G...",2019-11-01,zerohedge,"Enough ""Quid Pro Quo"" Gaslighting!",Horse trading is the oxygen of politics; it is...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 22:25:00 +0000",1572661500,1572648625,2
57154,"zerohedge--2019-11-01--""Born For This? I Don't...",2019-11-01,zerohedge,"""Born For This? I Don't Think So"" - Trump Mock...",Having plunged from over 10% to just 1% in the...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 21:56:35 +0000",1572659795,1572648626,2
57155,zerohedge--2019-11-01--How Iran Used Google To...,2019-11-01,zerohedge,How Iran Used Google To Disrupt 5% Of Global O...,Officials at Saudi Aramco believe that Iran us...,Tyler Durden,http://feedproxy.google.com/~r/zerohedge/feed/...,"Fri, 01 Nov 2019 21:45:00 +0000",1572659100,1572648627,2


# Keep only content, Reliability as columns

In [6]:
#Keep only content, Reliability as columns
totalData = totalData.drop(['id','date','source','title','author','url','published','published_utc','collection_utc'],axis=1)

In [7]:
totalData['Reliability'].value_counts()

0    44582
2    12575
Name: Reliability, dtype: int64

# Cleaning dataset

In [8]:
totalData = totalData.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
totalData.content = totalData.content.apply(clean_text)
totalData.content = totalData.content.str.replace('\d+', '')

In [9]:
totalData.sample(5)

,content,Reliability
4014,the use of bailiffs to pursue drivers for unpa...,0
18625,pakistans supreme court is due to decide on ja...,0
26496,if youve ever had misgivings about yoga adam f...,0
54528,this is what the liberal media thinks of ameri...,2
45274,as the world waits to see what the mueller inv...,2


In [10]:
X = totalData[['content']].to_numpy() #News content
Y = totalData[['Reliability']].to_numpy() #Labels

In [10]:
X

array([['last november stacey abrams a democrat who ran for governor of georgia came close to becoming the first black woman to lead a state in us history and when democrats announced she would take on the challenging task of rebutting president donald trumps state of the union address on feb  she once again came into the national spotlightabrams grew up in mississippi the daughter of a college librarian and a shipyard worker they eventually decided to become united methodist ministers picking up and moving the family of eight to georgia abramss family wasnt well off but she went on to build an impressive resume including degrees from spellman university and yale law schoolmy mom likes to call it the genteel poor we had no money but we watched pbs and we read books said abrams in an interview with abc news in january  they really believed that where we started out was not going to dictate where we ended upabrams rose through the political ranks eventually becoming minority leader in th

In [11]:
Y

array([[0],
       [0],
       [0],
       ...,
       [2],
       [2],
       [2]])

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/aayush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
def tokenize_and_transform(X, sample_size):
    essays1 = X[:, 0]
    #essays2 = X[:, 1]
    tok_es1 = [word_tokenize(doc) for doc in essays1[:sample_size]]
    #tok_es2 = [word_tokenize(doc) for doc in essays2[:sample_size]]
    met = MeanEmbeddingTransformer()
    X_transform =  met.fit_transform(tok_es1)
    return X_transform

In [13]:
X_transform = tokenize_and_transform(X, X.shape[0])


/home/aayush/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/aayush/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-3-2f4be04e443f>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([self._doc_mean(doc) for doc in X])


In [21]:
X_transform=X_transform.reshape(-1,1)

In [22]:
rus = RandomUnderSampler(random_state=0)
X_resample, y_resample = rus.fit_sample(X_transform, Y[:X_transform.shape[0]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_transform,
                                                    Y, test_size=0.2, random_state=0)

# Build model functions

In [16]:
def train_model(classifier, feature_vector_train, label,  feature_vector_valid, valid_y,is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    #print("In Validation Data",metrics.accuracy_score(predictions, valid_y))
    print("f1 score: ",f1_score(valid_y,predictions,average='weighted'))        
    return metrics.accuracy_score(valid_y,predictions)

In [25]:
def print_scores(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('F1 score: {:3f}'.format(f1_score(y_test, y_pred)))
    print('AUC score: {:3f}'.format(roc_auc_score(y_test, y_pred)))

# Naive Bayes

In [17]:
# Naive Bayes on Word Level TF IDF Vectors
print("For Glove vectors using Naive Bayes")
accuracy = train_model(naive_bayes.MultinomialNB(), X_train, y_train, X_test, y_test)
print ("Accuracy: ", accuracy)


For Glove vectors using Naive Bayes


ValueError: setting an array element with a sequence.

# load the pre-trained word-embedding vectors 


In [17]:
embeddings_dict = {}
with open("glove.6B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embeddings_dict[word] = vector

# create a tokenizer 


In [18]:
token = text.Tokenizer()
token.fit_on_texts(totalData['content'])
word_index = token.word_index

# Perform test train split

In [19]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['tweet'], train['label'])
train_x, valid_x, train_y, valid_y = train_test_split(totalData['content'], totalData['Reliability'], test_size=0.20)

# convert text to sequence of tokens and pad them to ensure equal length vectors 


In [30]:
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=1000)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=1000)

# create token-embedding mapping


In [32]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [34]:
embedding_matrix[3]

array([-7.69470036e-02, -2.12110002e-02,  2.12709993e-01, -7.22320020e-01,
       -1.39880002e-01, -1.22340001e-01, -1.75209999e-01,  1.21370003e-01,
       -7.08660036e-02, -1.57210004e+00, -2.24639997e-01,  4.26900014e-02,
       -4.01800007e-01,  2.10060000e-01,  1.42879998e-02,  4.16280001e-01,
        1.71649996e-02,  7.17319995e-02,  6.92459987e-03,  1.81070000e-01,
       -1.54119998e-01,  1.49330005e-01, -3.04930005e-02,  2.99180001e-01,
        2.94790007e-02, -3.61469984e-02, -6.11249991e-02,  8.39179978e-02,
       -1.23980001e-01, -1.00769997e-01, -5.41420002e-03,  3.37099999e-01,
       -2.56119996e-01,  4.43879992e-01, -6.89220011e-01,  1.80199996e-01,
        3.48980010e-01, -5.22839986e-02, -2.62259990e-01, -4.71089989e-01,
        2.16470003e-01, -4.00200009e-01, -4.99860011e-02,  1.13760000e-02,
        5.49939990e-01, -2.27909997e-01,  9.58729982e-02,  4.76929992e-01,
       -5.67269996e-02, -1.78949997e-01,  1.17559999e-01,  1.46620005e-01,
        4.89480011e-02,  